In [1]:
import argparse
import gc
import importlib
import os
import pickle
import random
import sys
import warnings
from copy import copy
import pandas as pd
import audiomentations as AA
import librosa
import numpy as np
import pandas as pd
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as torchdata
import torch_audiomentations as TAA
import albumentations

from sklearn import metrics
from timm.scheduler import CosineLRScheduler
from torch.cuda.amp import GradScaler, autocast
from torch.distributions import Beta
from torchaudio.transforms import AmplitudeToDB, MelSpectrogram
from tqdm import tqdm
from utils import AverageMeter

sys.path.append('../configs')
sys.path.append('./samplers')
sys.path.append('./pcen')
from pcen import StreamingPCENTransform
# from sampler import MultilabelBalancedRandomSampler

warnings.filterwarnings("ignore")
TRAIN_DATA_PATH = '../dataset/birdclef-2024/'

torch.set_flush_denormal(True)

/home/simon/miniconda3/envs/kaggle/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [24]:
ROOT = "/home/simon/disk1/Simon/Code/kaggle_competion_list/Birdclef/birdclef-2024/input"
sub = pd.read_csv(f"{ROOT}/birdclef-2024/sample_submission.csv")
target_columns = sub.columns.tolist()[1:]
num_classes = len(target_columns)
bird2id = {b: i for i, b in enumerate(target_columns)}
print(bird2id)
id2bird ={i:b for i, b in enumerate(target_columns)}


{'asbfly': 0, 'ashdro1': 1, 'ashpri1': 2, 'ashwoo2': 3, 'asikoe2': 4, 'asiope1': 5, 'aspfly1': 6, 'aspswi1': 7, 'barfly1': 8, 'barswa': 9, 'bcnher': 10, 'bkcbul1': 11, 'bkrfla1': 12, 'bkskit1': 13, 'bkwsti': 14, 'bladro1': 15, 'blaeag1': 16, 'blakit1': 17, 'blhori1': 18, 'blnmon1': 19, 'blrwar1': 20, 'bncwoo3': 21, 'brakit1': 22, 'brasta1': 23, 'brcful1': 24, 'brfowl1': 25, 'brnhao1': 26, 'brnshr': 27, 'brodro1': 28, 'brwjac1': 29, 'brwowl1': 30, 'btbeat1': 31, 'bwfshr1': 32, 'categr': 33, 'chbeat1': 34, 'cohcuc1': 35, 'comfla1': 36, 'comgre': 37, 'comior1': 38, 'comkin1': 39, 'commoo3': 40, 'commyn': 41, 'compea': 42, 'comros': 43, 'comsan': 44, 'comtai1': 45, 'copbar1': 46, 'crbsun2': 47, 'cregos1': 48, 'crfbar1': 49, 'crseag1': 50, 'dafbab1': 51, 'darter2': 52, 'eaywag1': 53, 'emedov2': 54, 'eucdov': 55, 'eurbla2': 56, 'eurcoo': 57, 'forwag1': 58, 'gargan': 59, 'gloibi': 60, 'goflea1': 61, 'graher1': 62, 'grbeat1': 63, 'grecou1': 64, 'greegr': 65, 'grefla1': 66, 'grehor1': 67, 'grej

In [28]:
col_index = [0, 6, 10, 11, 15, 18, 19, 20,
 24, 25, 26, 28, 30, 37, 38, 39, 40, 41, 44,
 45, 46, 47, 48, 49, 50, 53, 54, 57, 61, 62,
 64, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76,
 77, 78, 80, 82, 84, 87, 90, 91, 97, 100, 102,
 105, 107, 108, 109, 110, 111, 112, 113, 114,
 116, 117, 118, 119, 121, 123, 128, 129, 133,
 136, 137, 140, 141, 143, 146, 149, 153, 155,
 157, 161, 164, 165, 169, 173, 176, 177, 178, 180,
 181]
select_cv_cols = [id2bird[i] for i in  col_index]

In [3]:
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"


In [4]:
# https://github.com/issamemari/pytorch-multilabel-balanced-sampler/blob/master/sampler.py
import random

import numpy as np
from torch.utils.data.sampler import Sampler


class MultilabelBalancedRandomSampler(Sampler):
    """
    MultilabelBalancedRandomSampler: Given a multilabel dataset of length n_samples and
    number of classes n_classes, samples from the data with equal probability per class
    effectively oversampling minority classes and undersampling majority classes at the
    same time. Note that using this sampler does not guarantee that the distribution of
    classes in the output samples will be uniform, since the dataset is multilabel and
    sampling is based on a single class. This does however guarantee that all classes
    will have at least batch_size / n_classes samples as batch_size approaches infinity
    """

    def __init__(self, labels, indices=None, class_choice="least_sampled"):
        """
        Parameters:
        -----------
            labels: a multi-hot encoding numpy array of shape (n_samples, n_classes)
            indices: an arbitrary-length 1-dimensional numpy array representing a list
            of indices to sample only from
            class_choice: a string indicating how class will be selected for every
            sample:
                "least_sampled": class with the least number of sampled labels so far
                "random": class is chosen uniformly at random
                "cycle": the sampler cycles through the classes sequentially
        """
        self.labels = labels
        self.indices = indices
        if self.indices is None:
            self.indices = range(len(labels))

        self.num_classes = self.labels.shape[1]

        # List of lists of example indices per class
        self.class_indices = []
        for class_ in range(self.num_classes):
            lst = np.where(self.labels[:, class_] == 1)[0]
            lst = lst[np.isin(lst, self.indices)]
            self.class_indices.append(lst)

        self.counts = [0] * self.num_classes

        assert class_choice in ["least_sampled", "random", "cycle"]
        self.class_choice = class_choice
        self.current_class = 0

    def __iter__(self):
        self.count = 0
        return self

    def __next__(self):
        if self.count >= len(self.indices):
            raise StopIteration
        self.count += 1
        return self.sample()

    def sample(self):
        class_ = self.get_class()
        class_indices = self.class_indices[class_]
        chosen_index = np.random.choice(class_indices)
        if self.class_choice == "least_sampled":
            for class_, indicator in enumerate(self.labels[chosen_index]):
                if indicator == 1:
                    self.counts[class_] += 1
        return chosen_index

    def get_class(self):
        if self.class_choice == "random":
            class_ = random.randint(0, self.labels.shape[1] - 1)
        elif self.class_choice == "cycle":
            class_ = self.current_class
            self.current_class = (self.current_class + 1) % self.labels.shape[1]
        elif self.class_choice == "least_sampled":
            min_count = self.counts[0]
            min_classes = [0]
            for class_ in range(1, self.num_classes):
                if self.counts[class_] < min_count:
                    min_count = self.counts[class_]
                    min_classes = [class_]
                if self.counts[class_] == min_count:
                    min_classes.append(class_)
            class_ = np.random.choice(min_classes)
        return class_

    def __len__(self):
        return len(self.indices)


In [5]:
import copy  
import importlib.util  
import os  
  
# 假设你的配置文件是一个 Python 模块，并且它位于某个固定的路径下  
# 例如，配置文件名为 'config.py'，它位于与当前脚本相同的目录下  
CONFIG_FILE_NAME = 'exp071.py'  # 配置文件名  
CONFIG_MODULE_NAME = CONFIG_FILE_NAME.replace('.py', '')  # 转换为模块名  
  
# 获取当前脚本的目录（如果是从脚本运行的话）  
# 注意：如果是从模块导入的，这个可能不是你想要的目录  
CURRENT_DIR = "/home/simon/disk1/Simon/Code/kaggle_competion_list/Birdclef/birdclef-2024/BirdCLEF-2023-Identify-bird-calls-in-soundscapes-main-20240518T060118Z-001/BirdCLEF-2023-Identify-bird-calls-in-soundscapes-main/configs"
CONFIG_FILE_PATH = os.path.join(CURRENT_DIR, CONFIG_FILE_NAME)  
  
# 检查配置文件是否存在  
if not os.path.exists(CONFIG_FILE_PATH):  
    raise FileNotFoundError(f"Configuration file {CONFIG_FILE_PATH} not found.")  
  
# 使用importlib导入配置文件模块  
spec = importlib.util.spec_from_file_location(CONFIG_MODULE_NAME, CONFIG_FILE_PATH)  
config_module = importlib.util.module_from_spec(spec)  
spec.loader.exec_module(config_module)  
  
# 假设你的配置文件模块中有一个名为 'cfg' 的变量或属性  
CFG = copy.deepcopy(getattr(config_module, 'cfg', None))  
if CFG is None:  
    raise AttributeError(f"Module {CONFIG_MODULE_NAME} does not have an attribute 'cfg'.")  
  


In [6]:
# parser = argparse.ArgumentParser(description="")
# parser.add_argument("-C", "--config", help="config filename")
# parser_args, _ = parser.parse_known_args(sys.argv)
# CFG = copy(importlib.import_module(parser_args.config).cfg)


In [7]:


def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def get_device() -> torch.device:
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")


def init_logger(log_file='train.log'):
    from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


def load_wave_and_crop(filename, period, start=None):

    waveform_orig, sample_rate = librosa.load(filename, sr=32000, mono=False)

    wave_len = len(waveform_orig)
    waveform = np.concatenate([waveform_orig, waveform_orig, waveform_orig])

    effective_length = sample_rate * period
    while len(waveform) < (period * sample_rate * 3):
        waveform = np.concatenate([waveform, waveform_orig])
    if start is not None:
        start = start - (period - 5) / 2 * sample_rate
        while start < 0:
            start += wave_len
        start = int(start)
    else:
        if wave_len < effective_length:
            start = np.random.randint(effective_length - wave_len)
        elif wave_len > effective_length:
            start = np.random.randint(wave_len - effective_length)
        elif wave_len == effective_length:
            start = 0

    waveform_seg = waveform[start: start + int(effective_length)]

    return waveform_orig, waveform_seg, sample_rate, start


In [8]:
import torchvision
from torchvision.transforms import v2 as transforms


In [9]:
# for wave trasnform 1d  (160000,)
# tr_transforms = albumentations.Compose(
#     [
#     #    albumentations.OneOf([
#     #         AA.Gain(min_gain_in_db=-15, max_gain_in_db=15, p=1.0),
#     #         AA.GainTransition(
#     #             min_gain_in_db=-24.0,
#     #             max_gain_in_db=6.0,
#     #             min_duration=0.2,
#     #             max_duration=6.0,
#     #             p=1.0
#     #         )
#     #     ], p=0.5,),
#         albumentations.HorizontalFlip(p=0.5),
#         albumentations.CoarseDropout(max_height=int(128 * 0.375), max_width=int(128 * 0.375), max_holes=1, p=0.7),
#         # albumentations.XYMasking(
#         #         p=0.3,
#         #         num_masks_x=(1, 3),
#         #         num_masks_y=(1, 3),
#         #         mask_x_length=(1, 10),
#         #         mask_y_length=(1, 20),
#         #     ) ,
#     ]
# )

tr_transforms = AA.Compose(
    [
        AA.OneOf([
            AA.Gain(min_gain_in_db=-15, max_gain_in_db=15, p=1.0),
            AA.GainTransition(
                min_gain_in_db=-24.0,
                max_gain_in_db=6.0,
                min_duration=0.2,
                max_duration=6.0,
                p=1.0
            )
        ], p=0.5,),
        # AA.OneOf([
        #     AA.AddGaussianNoise(p=1.0),
        #     AA.AddGaussianSNR(p=1.0),
        # ], p=0.3,),
        # AA.OneOf([
        #     AA.AddShortNoises(
        #         sounds_path="../input/esc50/use_label",
        #         min_snr_in_db=0,
        #         max_snr_in_db=3,
        #         p=1.0,
        #         lru_cache_size=10,
        #         min_time_between_sounds=4.0,
        #         max_time_between_sounds=16.0,
        #     ),
        # ], p=0.5,),
        # AA.OneOf([
        #     AA.AddBackgroundNoise(
        #         sounds_path="../input/zenodo_nocall_30sec",
        #         min_snr_in_db=0,
        #         max_snr_in_db=3,
        #         p=1.0,
        #         lru_cache_size=1400,),
        # ], p=0.5,),
        AA.LowPassFilter(p=0.5),
    ]
)


# alb_transform = [
#         ### num_masks_x=1, num_masks_y=1 will change
#         albumentations.XYMasking(num_masks_x=2, num_masks_y=1, 
#                                  mask_x_length=20, mask_y_length=4,
#                                  fill_value=0, mask_fill_value=0, p=CFG.aug_spec_xymasking),
#         albumentations.CoarseDropout(fill_value=0, min_holes=20, max_holes=50, p=CFG.aug_spec_coarsedrop),
#         albumentations.HorizontalFlip(p=CFG.aug_spec_hflip),
#         # albumentations.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ]
# tr_transforms = albumentations.Compose(alb_transform)

# va_transform = AA.Compose(
#     # albumentations.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),1
# )


taa_augmentation = TAA.Compose(
    transforms=[
        TAA.PitchShift(
            sample_rate=CFG.sample_rate,
            mode="per_example",
            p=0.2,
            ),
    ]
)


In [10]:

class BirdClef2023Dataset(torchdata.Dataset):
    def __init__(
        self,
        data_path: str = 'DATA_PATH',
        period: float = 5.0,
        secondary_coef: float = 1.0,
        smooth_label: float = 0.05,
        df: pd.DataFrame = 'DATAFRAME',
        train: bool = True,
    ):

        self.df = df
        self.data_path = data_path
        self.filenames = df["filename"]

        self.primary_label = df["primary_label"]
        self.secondary_labels = (
            df["secondary_labels"]
            .map(
                lambda s: s.replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .replace("'", "")
                .split(" ")
            ).values
        )

        self.secondary_coef = secondary_coef
        # self.type = df["type"]
        self.teacher_preds = df["teacher_preds"]
        self.rating = df["rating"]
        self.period = period
        self.smooth_label = smooth_label + 1e-6
        self.wave_transforms = tr_transforms
        # self.wave_transforms_val = va_transforms
        self.train = train

    def __len__(self):
        return len(self.df)
   
    def normalize(self, x):
        valid_values = x[x != float('-inf')]
        mean_value = np.mean(valid_values)
        x[x == float('-inf')] = mean_value
        # x[x == float('-inf')] = 0

        x = x - x.min()
        x = x / x.max()
        return x

    def __getitem__(self, idx):

        filename = os.path.join(self.data_path, self.filenames[idx])

        if self.train:
            waveform, waveform_seg, sample_rate, start = load_wave_and_crop(
                filename, self.period ,0                                     # 训练也是前5s
            )
            # print(f"1----{waveform_seg.shape=}")
            # for i in range(len(waveform_seg )):
            #     waveform_seg[i] = self.normalize(waveform_seg[i])
            # print(type(waveform_seg))
            # waveform_seg = self.wave_transforms(
            #     # image = waveform_seg
            #     samples=waveform_seg, sample_rate=sample_rate
            #     )
            # waveform_seg = waveform_seg[np.newaxis,:]
            # print(waveform_seg.shape)

            res = self.wave_transforms(
                # image = waveform_seg
                samples=waveform_seg, sample_rate=sample_rate
                )
            # waveform_seg = res['image'].astype(np.float32)
            # # print("2",type(waveform_seg))
            # waveform_seg = waveform_seg.flatten()
            # print("2",waveform_seg.shape)


            

        else:
            waveform, waveform_seg, sample_rate, start = load_wave_and_crop(
                filename, self.period, 0
            )
            # for i in range(len(waveform_seg)):
            #     waveform_seg[i] = self.normalize(waveform_seg[i])
            # waveform_seg = self.wave_transforms_val(
            #     samples=waveform_seg, sample_rate=sample_rate
            #     )

        waveform_seg = torch.from_numpy(np.nan_to_num(waveform_seg)).float()


        rating = self.rating[idx]

        teacher_preds = torch.from_numpy(np.nan_to_num(self.teacher_preds[idx])).float()

        target = np.zeros(CFG.num_classes, dtype=np.float32)
        if self.primary_label[idx] != 'nocall':
            primary_label = CFG.bird2id[self.primary_label[idx]]
            target[primary_label] = 1.0

            if self.train:
                for s in self.secondary_labels[idx]:
                    if s != "" and s in CFG.bird2id.keys():
                        target[CFG.bird2id[s]] = self.secondary_coef

        target = torch.from_numpy(target).float()
        # teacher_preds = target

        return {
            "wave": waveform_seg,
            "rating": rating,
            "primary_targets": (target > 0.5).float(),
            "loss_target": target * (1-self.smooth_label) + self.smooth_label / target.size(-1),
            "teacher_preds": teacher_preds,
        }


def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1.0 / p)


class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = torch.nn.Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)


def gem_freq(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), 1)).pow(1.0 / p)


class GeMFreq(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super().__init__()
        self.p = torch.nn.Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        return gem_freq(x, p=self.p, eps=self.eps)


class NormalizeMelSpec(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.eps = eps

    def forward(self, X):
        mean = X.mean((1, 2), keepdim=True)
        std = X.std((1, 2), keepdim=True)
        Xstd = (X - mean) / (std + self.eps)
        norm_min, norm_max = \
            Xstd.min(-1)[0].min(-1)[0], Xstd.max(-1)[0].max(-1)[0]
        fix_ind = (norm_max - norm_min) > self.eps * torch.ones_like(
            (norm_max - norm_min)
        )
        V = torch.zeros_like(Xstd)
        if fix_ind.sum():
            V_fix = Xstd[fix_ind]
            norm_max_fix = norm_max[fix_ind, None, None]
            norm_min_fix = norm_min[fix_ind, None, None]
            V_fix = torch.max(
                torch.min(V_fix, norm_max_fix),
                norm_min_fix,
            )
            V_fix = (V_fix - norm_min_fix) / (norm_max_fix - norm_min_fix)
            V[fix_ind] = V_fix
        return V


class Mixup(nn.Module):
    def __init__(self, mix_beta):

        super(Mixup, self).__init__()
        self.beta_distribution = Beta(mix_beta, mix_beta)

    def forward(self, X, Y, weight=None, teacher_preds=None):

        bs = X.shape[0]
        n_dims = len(X.shape)
        perm = torch.randperm(bs)
        coeffs = self.beta_distribution.rsample(torch.Size((bs,))).to(X.device)

        if n_dims == 2:
            X = coeffs.view(-1, 1) * X + (1 - coeffs.view(-1, 1)) * X[perm]
        elif n_dims == 3:
            X = coeffs.view(-1, 1, 1) * X + (1 - coeffs.view(-1, 1, 1)) * X[perm]
        else:
            X = coeffs.view(-1, 1, 1, 1) * X + (1 - coeffs.view(-1, 1, 1, 1)) * X[perm]

        Y = coeffs.view(-1, 1) * Y + (1 - coeffs.view(-1, 1)) * Y[perm]

        if weight is None:
            return X, Y
        else:
            weight = coeffs.view(-1) * weight + (1 - coeffs.view(-1)) * weight[perm]
            teacher_preds = coeffs.view(-1, 1) * teacher_preds + (1 - coeffs.view(-1, 1)) * teacher_preds[perm]
            return X, Y, weight, teacher_preds


class AttModel(nn.Module):
    def __init__(
        self,
        backbone="resnet34",
        num_class=397,
        train_period=15.0,
        infer_period=5.0,
        in_chans=1,
    ):
        super().__init__()

        if CFG.use_pcen:
            self.pcen = StreamingPCENTransform(
                n_mels=CFG.n_mels,
                n_fft=CFG.n_fft,
                hop_length=CFG.hop_length,
                trainable=True,
                use_cuda_kernel=False
                )
            self.pcen.last_state = torch.zeros([55, 1, 128])
        else:
            self.logmelspec_extractor = nn.Sequential(
                MelSpectrogram(
                    sample_rate=CFG.sample_rate,
                    n_mels=CFG.n_mels,
                    f_min=CFG.fmin,
                    f_max=CFG.fmax,
                    n_fft=CFG.n_fft,
                    hop_length=CFG.hop_length,
                    normalized=True,
                ),
                AmplitudeToDB(top_db=80.0),
                NormalizeMelSpec(),
            )

        base_model = timm.create_model(
            backbone,
            features_only=False,
            pretrained=CFG.use_imagenet_weights,
            in_chans=CFG.in_channels,
        )

        layers = list(base_model.children())[:-2]
        self.backbone = nn.Sequential(*layers)
        if "efficientnet" in CFG.backbone:
            dense_input = base_model.num_features
        elif "swin" in CFG.backbone:
            dense_input = base_model.num_features
        elif hasattr(base_model, "fc"):
            dense_input = base_model.fc.in_features
        else:
            dense_input = base_model.feature_info[-1]["num_chs"]

        self.train_period = train_period
        self.infer_period = infer_period

        self.factor = int(self.train_period / self.infer_period)
        self.mixup = Mixup(mix_beta=1)
        self.global_pool = GeM()
        self.dropouts = nn.ModuleList([nn.Dropout(p) for p in np.linspace(0.1, 0.5, 5)])
        self.head = nn.Linear(dense_input, num_class)

    def forward(self, input):

        if self.training:
            x = input['wave']
            bs, time = x.shape
            x = x.reshape(bs * self.factor, time // self.factor)
            y = input["loss_target"]
            weight = input["rating"]
            teacher_preds = input["teacher_preds"]
        else:
            x = input['wave']
            y = input["loss_target"]
            weight = input["rating"]
            teacher_preds = input["teacher_preds"]

        if CFG.use_pcen:
            x = self.pcen(x).unsqueeze(1)
            self.pcen.reset()
        else:
            x = self.logmelspec_extractor(x)[:, None]

        if self.training:
            if np.random.random() <= 0.5:
                y2 = torch.repeat_interleave(y, self.factor, dim=0)
                weight2 = torch.repeat_interleave(weight, self.factor, dim=0)
                teacher_preds2 = torch.repeat_interleave(
                    teacher_preds, self.factor, dim=0
                    )

                for i in range(0, x.shape[0], self.factor):
                    x[i: i + self.factor], _, _, _ = self.mixup(
                        x[i: i + self.factor],
                        y2[i: i + self.factor],
                        weight2[i: i + self.factor],
                        teacher_preds2[i: i + self.factor],
                    )

            # # aug 
            # print(f"1---{x.shape=}")
            # result = tr_transforms(image=x)  
            # x= result["image"]


            b, c, f, t = x.shape
            x = x.permute(0, 3, 1, 2)
            # print(f"2---{x.shape=}")
            x = x.reshape(b // self.factor, self.factor * t, c, f)

            if np.random.random() <= CFG.mixup_p:
                x, y, weight, teacher_preds = self.mixup(x, y, weight, teacher_preds)

            x = x.reshape(b, t, c, f)
            x = x.permute(0, 2, 3, 1)

        x = self.backbone(x)

        if self.training:
            b, c, f, t = x.shape
            x = x.permute(0, 3, 1, 2)
            x = x.reshape(b // self.factor, self.factor * t, c, f)
            x = x.permute(0, 2, 3, 1)

        x = self.global_pool(x)
        x = x[:, :, 0, 0]
        logit = sum([self.head(dropout(x)) for dropout in self.dropouts]) / 5

        return {"logit": logit, "target": y, "rating": weight, "teacher_preds": teacher_preds}



In [11]:
def sigmoid_focal_loss(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    alpha: float = 0.25,
    gamma: float = 2,
    reduction: str = "none",
) -> torch.Tensor:
    """
    Loss used in RetinaNet for dense detection: https://arxiv.org/abs/1708.02002.

    Args:
        inputs (Tensor): A float tensor of arbitrary shape.
                The predictions for each example.
        targets (Tensor): A float tensor with the same shape as inputs. Stores the binary
                classification label for each element in inputs
                (0 for the negative class and 1 for the positive class).
        alpha (float): Weighting factor in range (0,1) to balance
                positive vs negative examples or -1 for ignore. Default: ``0.25``.
        gamma (float): Exponent of the modulating factor (1 - p_t) to
                balance easy vs hard examples. Default: ``2``.
        reduction (string): ``'none'`` | ``'mean'`` | ``'sum'``
                ``'none'``: No reduction will be applied to the output.
                ``'mean'``: The output will be averaged.
                ``'sum'``: The output will be summed. Default: ``'none'``.
    Returns:
        Loss tensor with the reduction option applied.
    """
    # Original implementation from https://github.com/facebookresearch/fvcore/blob/master/fvcore/nn/focal_loss.py


    p = torch.sigmoid(inputs)
    ce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction="none")
    p_t = p * targets + (1 - p) * (1 - targets)
    loss = ce_loss * ((1 - p_t) ** gamma)

    if alpha >= 0:
        alpha_t = alpha * targets + (1 - alpha) * (1 - targets)
        loss = alpha_t * loss

    # Check reduction option and return loss accordingly
    if reduction == "none":
        pass
    elif reduction == "mean":
        loss = loss.mean()
    elif reduction == "sum":
        loss = loss.sum()
    else:
        raise ValueError(
            f"Invalid Value for arg 'reduction': '{reduction} \n Supported reduction modes: 'none', 'mean', 'sum'"
        )
    return loss

In [27]:

class BCEKDLoss(nn.Module):
    def __init__(self, weights=[0.1, 0.9], class_weights=None):
        super().__init__()

        self.weights = weights
        self.T = 20

    def forward(self, output):
        input_ = output["logit"]
        target = output["target"].float()
        rating = output["rating"]
        teacher_preds = output["teacher_preds"]

        rating = rating.unsqueeze(1).repeat(1, CFG.num_classes)
        loss = nn.BCEWithLogitsLoss(
            weight=rating,
            reduction='mean',
        )(input_, target)

        KD_loss = nn.KLDivLoss()(
            F.log_softmax(input_ / self.T, dim=1),
            F.softmax(teacher_preds / self.T, dim=1)
            ) * (self.weights[1] * self.T * self.T)

        return self.weights[0] * loss + KD_loss


class FocalLossBCE(torch.nn.Module):
    def __init__(
            self,
            alpha: float = 0.25,
            gamma: float = 2,
            reduction: str = "mean",
            bce_weight: float = 1.0,
            focal_weight: float = 1.0,
    ):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.bce = torch.nn.BCEWithLogitsLoss(reduction=reduction)
        self.bce_weight = bce_weight
        self.focal_weight = focal_weight

    # def forward(self, logits, targets):
    def forward(self,output):
        logits = output["logit"]
        targets = output["target"].float()
        rating = output["rating"]
        teacher_preds = output["teacher_preds"]

        focall_loss = sigmoid_focal_loss(
            inputs=logits,
            targets=targets,
            alpha=self.alpha,
            gamma=self.gamma,
            reduction=self.reduction,
        )
        bce_loss = self.bce(logits, targets.float())
        return self.bce_weight * bce_loss + self.focal_weight * focall_loss



def padded_cmap(solution, submission, padding_factor=5):
    solution = solution  # .drop(['row_id'], axis=1, errors='ignore')
    submission = submission  # .drop(['row_id'], axis=1, errors='ignore')
    new_rows = []
    for i in range(padding_factor):
        new_rows.append([1 for i in range(len(solution.columns))])
    new_rows = pd.DataFrame(new_rows)
    new_rows.columns = solution.columns
    padded_solution = pd.concat(
        [solution, new_rows]).reset_index(drop=True).copy()
    padded_submission = pd.concat(
        [submission, new_rows]).reset_index(drop=True).copy()
    score = metrics.average_precision_score(
        padded_solution.values,
        padded_submission.values,
        average='macro',
    )
    return score


def map_score(solution, submission):
    solution = solution  # .drop(['row_id'], axis=1, errors='ignore')
    submission = submission  # .drop(['row_id'], axis=1, errors='ignore')
    score = metrics.average_precision_score(
        solution.values,
        submission.values,
        average='micro',  # 'macro'
    )
    return score


def batch_to_device(batch, device):
    batch_dict = {key: batch[key].to(device) for key in batch}
    return batch_dict


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    '''
    Version of macro-averaged ROC-AUC score that ignores all classes that have no true positive labels.
    '''
    del solution[row_id_column_name]
    del submission[row_id_column_name]

    solution_sums = solution.sum(axis=0)
    scored_columns = list(solution_sums[solution_sums > 0].index.values)
#     assert len(scored_columns) > 0

    return sklearn.metrics.roc_auc_score(solution[scored_columns].values, submission[scored_columns].values, average='macro')

def calculate_competition_metrics(gt, preds, target_columns, one_hot=True):
    if not one_hot:
        ground_truth = np.argmax(gt, axis=1)
        gt = np.zeros((ground_truth.size, len(target_columns)))
        gt[np.arange(ground_truth.size), ground_truth] = 1
    val_df = pd.DataFrame(gt, columns=target_columns)
    pred_df = pd.DataFrame(preds, columns=target_columns)
    # cmAP_1 = padded_cmap(val_df, pred_df, padding_factor=1)
    # cmAP_5 = padded_cmap(val_df, pred_df, padding_factor=5)
    val_df['id'] = [f'id_{i}' for i in range(len(val_df))]
    pred_df['id'] = [f'id_{i}' for i in range(len(pred_df))]
    train_score = score(val_df, pred_df, row_id_column_name='id')
    return {
    #   "cmAP_1": cmAP_1,
    #   "cmAP_5": cmAP_5,
      "ROC": train_score,
    }

def train_fn(data_loader, model, criterion, optimizer, scheduler, epoch):

    model.train()
    losses = AverageMeter()
    optimizer.zero_grad(set_to_none=True)
    scaler = GradScaler(enabled=CFG.apex)
    iters = len(data_loader)
    gt = []
    preds = []

    with tqdm(enumerate(data_loader), total=len(data_loader)) as t:

        for i, (data) in t:
            inputs = batch_to_device(data, device)
            targets = data['primary_targets'].to(device)

            inputs['wave'] = taa_augmentation(inputs['wave'].unsqueeze(1))
            inputs['wave'] = inputs['wave'].squeeze(1)

            with autocast(enabled=CFG.apex):
                outputs = model(inputs)
                loss = criterion(outputs)

            losses.update(loss.item(), inputs['wave'].size(0))

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            grad_norm = torch.nn.utils.clip_grad_norm_(
                model.parameters(), max_norm=CFG.max_grad_norm)
            scaler.step(optimizer)
            scaler.update()

            optimizer.zero_grad(set_to_none=True)
            scheduler.step(epoch + i / iters)
            t.set_postfix(
                loss=losses.avg,
                grad=grad_norm.item(),
                lr=optimizer.param_groups[0]["lr"]
                )

            gt.append(targets.cpu().detach().numpy())
            preds.append(outputs["logit"].sigmoid().cpu().detach().numpy())

    val_df = pd.DataFrame(
        np.concatenate(gt), columns=CFG.target_columns[:182])
    pred_df = pd.DataFrame(
        np.concatenate(preds), columns=CFG.target_columns[:182])
    cmAP_1 = padded_cmap(val_df, pred_df, padding_factor=1)
    cmAP_5 = padded_cmap(val_df, pred_df, padding_factor=5)
    mAP = map_score(val_df, pred_df)

    gt = np.concatenate(gt)
    preds = np.concatenate(preds)
    # print(gt, preds.shape, target_columns)
    gt = np.array(gt, dtype=np.int32)
    scores = calculate_competition_metrics(gt, preds, target_columns)


    return losses.avg, cmAP_1, cmAP_5, mAP, scores


import sklearn


def valid_fn(data_loader, model, criterion, epoch,col_index):
    model.eval()
    losses = AverageMeter()
    gt = []
    preds = []

    with tqdm(enumerate(data_loader), total=len(data_loader)) as t:
        for i, (data) in t:
            inputs = batch_to_device(data, device)
            targets = data['primary_targets'].to(device)

            with autocast(enabled=CFG.apex):
                with torch.no_grad():
                    outputs = model(inputs)
                    loss = criterion(outputs)

            losses.update(loss.item(), inputs['wave'].size(0))
            t.set_postfix(loss=losses.avg)

            gt.append(targets.cpu().detach().numpy())
            preds.append(outputs["logit"].sigmoid().cpu().detach().numpy())

    val_df = pd.DataFrame(np.concatenate(gt), columns=CFG.target_columns[:182])
    pred_df = pd.DataFrame(np.concatenate(preds), columns=CFG.target_columns[:182])
    cmAP_1 = padded_cmap(val_df, pred_df, padding_factor=1)
    cmAP_5 = padded_cmap(val_df, pred_df, padding_factor=5)
    mAP = map_score(val_df, pred_df)

    gt = np.concatenate(gt)
    preds = np.concatenate(preds)

    # print(gt, preds.shape, target_columns)
    gt = np.array(gt, dtype=np.int32)
    scores = calculate_competition_metrics(gt, preds, target_columns)

    target_columns =  [id2bird[i] for i in col_index]
    scores_cv = calculate_competition_metrics(gt[:,col_index], preds[:,col_index], target_columns)
    

    return losses.avg, cmAP_1, cmAP_5, mAP,scores,scores_cv


In [13]:

def train_loop(
    train_loader,
    val_loader,
    model,
    optimizer,
    scheduler,
    criterion,
    epochs=10,
    fold=None,
):

    best_score = 0.0
    patience = CFG.early_stopping
    n_patience = 0

    for epoch in range(epochs): 
        # train for one epoch
        train_loss, train_cmAP1, train_cmAP5, train_mAP,train_scores = train_fn(
            train_loader, model, criterion, optimizer, scheduler, epoch)

        # evaluate on validation set
        val_loss, val_cmAP1, val_cmAP5, val_mAP,scores = valid_fn(
            val_loader, model, criterion, epoch,col_index)
        
        train_score = train_scores["ROC"]
        val_score = scores["ROC"]
        logger.info(f"Epoch {epoch} - Train loss: {train_loss:.4f}, Train cmAP1: {train_cmAP1:.4f}, Train cmAP5: {train_cmAP5:.4f}, Train mAP: {train_mAP:.4f}, \n Valid loss: {val_loss:.4f}, Valid cmAP1: {val_cmAP1:.4f}, Valid cmAP5: {val_cmAP5:.4f}, Valid mAP: {val_mAP:.4f}")
        logger.info(f"\n Epoch {epoch} - train metric {train_score} val metirc {val_score}")


        is_better = val_score > best_score
        best_score = max(val_score, best_score)

        # Save the best model
        if is_better:
            state = {
                "epoch": epoch,
                "state_dict": model.state_dict(),
                "best_loss": best_score,
                "optimizer": optimizer.state_dict(),
                # "macro-roc-auc":scores["ROC"]
            }
            logger.info(
                f"Epoch {epoch} - Save Best Score: {best_score:.4f} Model\n")
            torch.save(
                state,
                os.path.join(CFG.model_output_path, f"fold_{fold}_model.bin")
                )
            
            # # 将模型转换为TorchScript
            # scripted_model = torch.jit.script(model)
            # # 保存序列化的模型
            # torch.jit.save(scripted_model, f'jit_model/fold_{fold}_model.pt')

            n_patience = 0
        else:
            n_patience += 1
            logger.info(
                f"Valid loss didn't improve last {n_patience} epochs.\n")

        if n_patience >= patience:
            logger.info(
                "Early stop, Training End.\n")
            break

    return


logger = init_logger(log_file=f"../log/train_{CFG.exp_name}.log")
device = get_device()
set_seed(CFG.seed)
os.makedirs(os.path.join(CFG.model_output_path), exist_ok=True)


In [14]:
# a = pd.read_csv("/home/simon/disk1/Simon/Code/kaggle_competion_list/Birdclef/birdclef-2024/BirdCLEF-2023-Identify-bird-calls-in-soundscapes-main-20240518T060118Z-001/BirdCLEF-2023-Identify-bird-calls-in-soundscapes-main/train_folds_0518.csv")

In [15]:
# # a["filename"] = a["filename"].apply(lambda x: x.replace("../dataset",""))
# a["filename"] = a["filename"].apply(lambda x: "/home/simon/disk1/Simon/Code/kaggle_competion_list/Birdclef/birdclef-2024/input"+x)


In [16]:
# a.to_csv("/home/simon/disk1/Simon/Code/kaggle_competion_list/Birdclef/birdclef-2024/BirdCLEF-2023-Identify-bird-calls-in-soundscapes-main-20240518T060118Z-001/BirdCLEF-2023-Identify-bird-calls-in-soundscapes-main/train_folds_0518.csv")

In [17]:
model = AttModel(
        backbone=CFG.backbone,
        num_class=CFG.num_classes,
        train_period=CFG.period,
        infer_period=5,
    )
# modelpath = "/home/simon/disk1/Simon/Code/kaggle_competion_list/Birdclef/birdclef-2024/BirdCLEF-2023-Identify-bird-calls-in-soundscapes-main-20240518T060118Z-001/BirdCLEF-2023-Identify-bird-calls-in-soundscapes-main/models/exp071_tf_efficientnet_b0_ns/fold_0_model.bin"
# len(torch.load(modelpath)["state_dict"].keys())

#363

In [18]:
external = pd.read_csv("/home/simon/disk1/Simon/Code/kaggle_competion_list/Birdclef/birdclef-2024/Train mnasnet/nocall_pseudo.csv")
external

,Unnamed: 0,primary_label,secondary_labels,filename,teacher_preds,teacher_label
0,0,nocall,[],64486.ogg,[ -6.8236113 -5.626453 -7.5165544 -9.73582...,grehor1
1,1,nocall,[],2525.ogg,[ -7.176402 -9.332384 -8.316804 -10.27886...,zitcis1
2,2,nocall,[],44981.ogg,[-4.0132084 -5.802989 -6.256181 -6.773272 -...,categr
3,3,nocall,[],101323.ogg,[-6.338834 -5.7509503 -7.5665483 -9.063977 -...,comgre
4,4,nocall,[],165746.ogg,[ -7.743705 -7.3720293 -9.064224 -10.81405...,eurcoo
...,...,...,...,...,...,...
5750,5750,nocall,[],95027.ogg,[ -6.1317134 -7.55363 -6.77861 -8.95025...,purher1
5751,5751,nocall,[],80708.ogg,[ -6.922803 -7.7936735 -7.4676466 -10.89...,rocpig
5752,5752,nocall,[],168059.ogg,[ -6.418939 -8.418465 -7.230104 -9.64820...,commoo3
5753,5753,nocall,[],164922.ogg,[ -5.4923444 -6.9318366 -7.868212 -9.33819...,graher1


In [19]:
external = pd.read_csv("/home/simon/disk1/Simon/Code/kaggle_competion_list/Birdclef/birdclef-2024/Train mnasnet/nocall_pseudo.csv")
external["filename"] = external["filename"].apply(lambda x: "/home/simon/disk1/Simon/Code/kaggle_competion_list/Birdclef/birdclef-2024/input/nocall_archive/ff1010bird_nocall/nocall"+x)
# external["type"] = '[]'
external["kfold"] = -1
external["rating"] = 1
select_cols = ['filename', 'primary_label','secondary_labels','kfold', 'rating',"teacher_preds"]
# external["filename"] = '../input/birdclef-2023/train_audio/' + 'nocall/' + external['filename']
# del external['length']

In [20]:
def str_array2np(s):
    # 去除字符串两端的方括号和换行符，并按空格分割  
    numbers_str = s.strip('[]\n').split()  
    # 将字符串列表转换为浮点数列表  
    numbers_float = [float(n) for n in numbers_str]  
    # 将浮点数列表转换为NumPy数组  
    numbers_array = np.array(numbers_float)  

    return numbers_array

In [21]:

# with open(TRAIN_DATA_PATH + 'train_meta_pseudo.pickle', 'rb') as f:
#     train = pickle.load(f)
# train['filename'] = '../input/birdclef-2023/train_audio/' + train['filename']
## 无teacher 训练
# train = pd.read_csv("../train_folds_0518.csv")
train = pd.read_csv("/home/simon/disk1/Simon/Code/kaggle_competion_list/Birdclef/birdclef-2024/Train mnasnet/trainfold_0518_pseudo.csv")
train['rating'] = 1
train["teacher_preds"] = train[train["kfold"]!=-1]["teacher_preds"].apply(lambda x :str_array2np(x))
# with open('../input/xeno-canto_audio_meta_pseudo.pickle', 'rb') as f:
#     xeno_canto_sa = pickle.load(f)
# xeno_canto_sa["fold"] = -1
# train = pd.concat([train, xeno_canto_sa]).reset_index(drop=True)

# with open('../input/xeno-canto_nd_audio_meta_pseudo.pickle', 'rb') as f:
#     xeno_canto_nd = pickle.load(f)
# train = pd.concat([train, xeno_canto_nd]).reset_index(drop=True)

if CFG.external:
    # with open('/home/simon/disk1/Simon/Code/kaggle_competion_list/Birdclef/birdclef-2024/input/ff1010bird_metadata_v1_pseudo.pickle', 'rb') as f:
    #     external = pickle.load(f)
    external = pd.read_pickle("/home/simon/disk1/Simon/Code/kaggle_competion_list/Birdclef/birdclef-2024/input/ff1010bird_metadata_v1_pseudo.pickle")
    # external = pd.read_csv("/home/simon/disk1/Simon/Code/kaggle_competion_list/Birdclef/birdclef-2024/Train mnasnet/nocall_pseudo.csv")
    external["filename"] = external["filename"].apply(lambda x: "/home/simon/disk1/Simon/Code/kaggle_competion_list/Birdclef/birdclef-2024/input/nocall_archive/ff1010bird_nocall/nocall"+x)
    # external["type"] = '[]'
    external["kfold"] = -1
    external["rating"] = 1
    select_cols = ['filename', 'primary_label','secondary_labels','kfold', 'rating',"teacher_preds"]
    # del external['length']
    
    print(external.shape)
    external.head()
    train["filename"] = train["path"]
    train = pd.concat([train[select_cols], external[select_cols]]).reset_index(drop=True)

# train["rating"] = np.clip(train["rating"] / train["rating"].max(), 0.1, 1.0)

logger.info(train.shape)
train.head()

# main loop
for fold in range(5):

    if fold not in CFG.folds:
        continue
    logger.info("=" * 90)
    logger.info(f"Fold {fold} Training")
    logger.info("=" * 90)

    trn_df = train[train['kfold'] != fold].reset_index(drop=True)
    val_df = train[train['kfold'] == fold].reset_index(drop=True)

    sampler = None
    if CFG.use_sampler:
        one_hot_target = np.zeros(
            (trn_df.shape[0], len(CFG.target_columns)), dtype=np.float32
            )

        for i, label in enumerate(trn_df.primary_label):
            if label != "nocall":
                primary_label = CFG.bird2id[label]
                one_hot_target[i, primary_label] = 1.0

        sampler = MultilabelBalancedRandomSampler(
            one_hot_target,
            trn_df.index,
            class_choice="least_sampled"
            )

    logger.info(trn_df.shape)
    logger.info(trn_df['primary_label'].value_counts())
    logger.info(val_df.shape)
    logger.info(val_df['primary_label'].value_counts())

    loaders = {}
    trn_dataset = BirdClef2023Dataset(
            data_path=CFG.train_datadir,
            period=CFG.period,
            secondary_coef=CFG.secondary_coef,
            train=True,
            df=trn_df,
    )
    loaders['train'] = torchdata.DataLoader(
        trn_dataset,
        sampler=sampler,
        **CFG.loader_params['train']
    )
    val_dataset = BirdClef2023Dataset(
            data_path=CFG.train_datadir,
            period=5,
            secondary_coef=CFG.secondary_coef,
            train=False,
            df=val_df,
    )
    loaders['valid'] = torchdata.DataLoader(
        val_dataset,
        **CFG.loader_params['valid']
    )

    model = AttModel(
        backbone=CFG.backbone,
        num_class=CFG.num_classes,
        train_period=CFG.period,
        infer_period=5,
    )

    if CFG.pretrained_weights:
        model_path = CFG.pretrained_path
        checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
        del checkpoint['state_dict']['head.weight']
        del checkpoint['state_dict']['head.bias']
        model.load_state_dict(checkpoint["state_dict"], strict=False)

        params = list(model.parameters())
        print('the length of parameters is', len(params))
        for i in range(len(params)):
            params[i].data = torch.round(params[i].data*10**17) / 10**17
        del checkpoint

    model = model.to(device)

    # del xeno_canto_sa, xeno_canto_nd, external
    gc.collect()

    # criterion = BCEKDLoss()
    criterion = FocalLossBCE()

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=CFG.lr_max,
        betas=(0.9, 0.999),
        eps=1e-08,
        weight_decay=CFG.weight_decay,
        amsgrad=False,
        )
    scheduler = CosineLRScheduler(
        optimizer,
        t_initial=10,
        warmup_t=1,
        cycle_limit=40,
        cycle_decay=1.0,
        lr_min=CFG.lr_min,
        t_in_epochs=True,
    )

    # start training
    train_loop(
        loaders['train'],
        loaders['valid'],
        model,
        optimizer,
        scheduler,
        criterion,
        epochs=CFG.epochs,
        fold=fold,
        )

logger.info('training done!')


(54310, 6)
Fold 0 Training
(44599, 6)
primary_label
nocall     5755
houspa     2078
eaywag1    1977
commoo3    1766
eurcoo     1701
           ... 
blaeag1       9
integr        8
wbbfly1       7
niwpig1       6
asiope1       5
Name: count, Length: 183, dtype: int64
(9711, 6)
primary_label
houspa     520
eaywag1    494
commoo3    441
eurcoo     426
barswa     345
          ... 
niwpig1      2
asiope1      2
blaeag1      2
wbbfly1      2
malwoo1      2
Name: count, Length: 182, dtype: int64


(5755, 7)
the length of parameters is 214


100%|██████████| 152/152 [00:54<00:00,  2.78it/s, loss=0.0124]
Epoch 0 - Train loss: 0.0856, Train cmAP1: 0.0702, Train cmAP5: 0.0910, Train mAP: 0.0577, 
 Valid loss: 0.0124, Valid cmAP1: 0.7426, Valid cmAP5: 0.8176, Valid mAP: 0.8228

 Epoch 0 - train metric 0.7213252864457123 val metirc 0.969718827117269
Epoch 0 - Save Best Score: 0.9697 Model

100%|██████████| 152/152 [00:54<00:00,  2.81it/s, loss=0.012] 
Epoch 1 - Train loss: 0.0168, Train cmAP1: 0.3286, Train cmAP5: 0.3514, Train mAP: 0.3154, 
 Valid loss: 0.0120, Valid cmAP1: 0.7418, Valid cmAP5: 0.8184, Valid mAP: 0.8243

 Epoch 1 - train metric 0.8447800862522313 val metirc 0.9732892060784089
Epoch 1 - Save Best Score: 0.9733 Model

100%|██████████| 152/152 [00:50<00:00,  3.03it/s, loss=0.012] 
Epoch 2 - Train loss: 0.0162, Train cmAP1: 0.3303, Train cmAP5: 0.3533, Train mAP: 0.3173, 
 Valid loss: 0.0120, Valid cmAP1: 0.7383, Valid cmAP5: 0.8185, Valid mAP: 0.8243

 Epoch 2 - train metric 0.851772001878473 val metirc 0.9744766

In [22]:
train[train["kfold"]==0]["teacher_preds"].iloc[0]


array([-2.3632386, -6.221536 , -5.1751814, -6.8329625, -5.520384 ,
       -8.106002 , -6.291426 , -6.7113867, -5.6586127, -4.2065187,
       -5.577632 , -7.779959 , -6.7608037, -6.14487  , -6.1412306,
       -5.894186 , -8.003578 , -4.298207 , -4.67443  , -5.2208414,
       -3.7749233, -6.8253655, -6.660148 , -6.213027 , -5.4677286,
       -7.8196154, -7.110083 , -5.468467 , -5.6851654, -6.8851123,
       -6.9113493, -6.502802 , -5.0228243, -7.3172398, -7.6176114,
       -6.912636 , -6.88361  , -5.577966 , -5.2653103, -2.5930033,
       -4.478192 , -3.698809 , -6.9878225, -4.5013027, -4.5943346,
       -5.836675 , -5.708394 , -4.4435215, -4.9604793, -7.8057895,
       -5.3005514, -6.7376738, -7.9047747, -3.5978856, -6.7750597,
       -4.302254 , -6.631811 , -4.206804 , -4.163726 , -7.1966066,
       -7.090315 , -6.7487826, -4.3251696, -4.581149 , -5.4554977,
       -6.8678255, -7.224822 , -8.301522 , -6.623717 , -7.6582403,
       -2.338531 , -4.30272  , -4.5604377, -4.845799 , -7.1887

In [23]:
train[train["kfold"]==-1]["teacher_preds"].iloc[0]

array([-12.8047085,  -9.238798 ,  -9.28768  , -12.805975 , -13.4300995,
       -12.065721 , -11.905288 , -15.5233555, -11.37521  , -10.584167 ,
        -7.880215 , -11.568201 ,  -8.751513 , -10.0683975, -10.57903  ,
       -15.038082 ,  -9.928907 , -10.742993 , -13.881045 , -13.013288 ,
        -7.819019 ,  -8.921952 , -15.999883 ,  -8.188779 , -14.371161 ,
        -9.523504 , -13.702866 , -13.303877 , -11.960668 ,  -8.356654 ,
        -9.913391 ,  -9.508785 , -12.31569  , -11.566313 , -12.6911125,
       -14.268026 , -12.118282 , -12.02198  , -10.839478 , -13.68715  ,
       -13.477827 , -14.657263 , -12.701844 , -13.337585 , -16.851261 ,
       -12.766783 , -13.6003   , -11.453594 , -13.125055 , -13.31653  ,
       -12.183271 , -13.693626 , -13.461819 , -12.994865 , -12.466036 ,
       -10.318794 , -10.683501 , -11.121885 , -12.346202 , -14.208176 ,
        -9.299595 , -10.820049 ,  -9.307911 , -14.914533 , -16.925846 ,
       -13.67597  , -10.911665 , -13.121988 , -12.529295 , -11.1